In [1]:
import dataset
import utils
import reference_impl as ref
import predictors
import model

# Use this cfg variable whenever we need to access some constant
cfg = utils.Config(
    num_rx_antennas=1,
    num_tx_antennas=32,
    num_subcarriers=80,
    train_test_split=0.8,
    data_root="../data/dataset1",
    reduce_pca_overhead=False
    # duplicate_data=1,
    # data_snr=-1
)

train_set = dataset.dataset_from_path("../data/dataset1/1Lane_2014U_1Rx_32Tx_1T_80K_2620000000.0fc.pickle", cfg)

In [2]:
## Offline Training
import matlab.engine

matlab = matlab.engine.start_matlab()
pca = ref.ReferencePCA(cfg, matlab)
model.train_or_load(pca, cfg.pca_path, cfg.retrain_all, train_set.csi_samples)  # pca.fit() includes reduce overhead
zdl_train = pca.process(train_set.csi_samples)  # N * zdl_len

In [3]:
## Train Predictor
predictor = predictors.VARPredictor(cfg)
model.train_or_load(predictor, cfg.predictor_path, cfg.retrain_all, zdl_train)

In [4]:
predicted_zdl = predictor.process(zdl_train)
prediction_error = zdl_train - predicted_zdl

ValueError: operands could not be broadcast together with shapes (2009,2008) (1,2009) 

In [ ]:
## Error Compression
error_compressor = ref.ReferenceKmeans(cfg, matlab)
model.train_or_load(error_compressor, cfg.kmeans_path, cfg.retrain_all, prediction_error)

# DL and UL each get trained pca, predictor, and error_compressor